In [1]:
import os
import glob
import netCDF4
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib import colors
import cartopy.crs as ccrs
import cartopy.feature
import seaborn as sns

import wrf
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import matplotlib.ticker as mticker
import matplotlib.colors

In [ ]:
######################
# FUNCTIONS
######################

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
######################
# ANT
#Parameters
folder_mod = '/capstor/scratch/cscs/gsergi/OUTPUT_PGW_20220315/ANT_20220315_'
domain = 'd01'
nlayers = 100
acc_t = 1440 #minutes
ini_date_idx=0
#end_date_idx=0

###load data###
snpack_files = '/snpack/snowpack_'+domain+'_*'# Load netcdf
outhist_files = '/outhist/outhist_'+domain+'_*'# Load netcdf

################
#hist
print('Charging Ant hist...')
wrflist_sp = []
proj = 'hist'
print('Total: ' + str(np.size(glob.glob(folder_mod+proj+snpack_files))))
for i,ncdf_oh in enumerate(np.sort(glob.glob(folder_mod+proj+snpack_files))):
        print(i, end="\r")
        wrflist_sp.append(netCDF4.Dataset(ncdf_oh))
    
# Get surface variables
sp_VolI_hist = wrf.getvar(wrflist_sp, 'SN_VOLI', timeidx=wrf.ALL_TIMES)[ini_date_idx:,:nlayers,:,:]
sp_VolW_hist = wrf.getvar(wrflist_sp, 'SN_VOLW', timeidx=wrf.ALL_TIMES)[ini_date_idx:,:nlayers,:,:]
x =  wrf.getvar(wrflist_sp, 'SN_THICK', timeidx=wrf.ALL_TIMES)[:,:nlayers+1,:,:].cumsum(axis=1)
sp_depth_hist = (x[:,1:,:,:] + x[:,:-1,:,:]) / 2

#hist
wrflist_oh = []
proj = 'hist'
print('Total: ' + str(np.size(glob.glob(folder_mod+proj+outhist_files))))
for i,ncdf_oh in enumerate(np.sort(glob.glob(folder_mod+proj+outhist_files))):
        print(i, end="\r")
        wrflist_oh.append(netCDF4.Dataset(ncdf_oh))
    
# Get surface variables
t2_ant_hist = wrf.getvar(wrflist_oh, 'T2', timeidx=wrf.ALL_TIMES)[ini_date_idx:,:,:]

################
#past
print('Charging Ant past...')
wrflist_sp = []
proj = 'past_new'
print('Total: ' + str(np.size(glob.glob(folder_mod+proj+snpack_files))))
for i,ncdf_oh in enumerate(np.sort(glob.glob(folder_mod+proj+snpack_files))):
        print(i, end="\r")
        wrflist_sp.append(netCDF4.Dataset(ncdf_oh))
    
# Get surface variables
sp_VolI_past = wrf.getvar(wrflist_sp, 'SN_VOLI', timeidx=wrf.ALL_TIMES)[ini_date_idx:,:nlayers,:,:]
sp_VolW_past = wrf.getvar(wrflist_sp, 'SN_VOLW', timeidx=wrf.ALL_TIMES)[ini_date_idx:,:nlayers,:,:]
x =  wrf.getvar(wrflist_sp, 'SN_THICK', timeidx=wrf.ALL_TIMES)[:,:nlayers+1,:,:].cumsum(axis=1)
sp_depth_past = (x[:,1:,:,:] + x[:,:-1,:,:]) / 2

#hist
wrflist_oh = []
proj = 'past_new'
print('Total: ' + str(np.size(glob.glob(folder_mod+proj+outhist_files))))
for i,ncdf_oh in enumerate(np.sort(glob.glob(folder_mod+proj+outhist_files))):
        print(i, end="\r")
        wrflist_oh.append(netCDF4.Dataset(ncdf_oh))
    
# Get surface variables
t2_ant_past = wrf.getvar(wrflist_oh, 'T2', timeidx=wrf.ALL_TIMES)[ini_date_idx:,:,:]

######################
# ANT COORDINATES
# Get surface fix variables
height = wrf.getvar(wrflist_sp, 'HGT', timeidx=0, method="cat")[:,:]
# Get dimension
lats, lons = wrf.latlon_coords(sp_VolI_hist)
times = wrf.extract_times(wrflist_sp, timeidx=wrf.ALL_TIMES)

Charging Ant hist...
Total: 59
Total: 59
Charging Ant past...
Total: 59


In [4]:
#Positions
CON_xy = wrf.ll_to_xy(wrflist_sp, -75.09978, 123.332196) #Concordia
CON_ll = wrf.xy_to_ll(wrflist_sp, CON_xy[0], CON_xy[1])
DDU_xy = wrf.ll_to_xy(wrflist_sp, -66.662778, 140.001111) #DDU
DDU_ll = wrf.xy_to_ll(wrflist_sp, DDU_xy[0], DDU_xy[1])
CIS_xy = wrf.ll_to_xy(wrflist_sp, -66.033333, 103.55) #DDU
CIS_ll = wrf.xy_to_ll(wrflist_sp, CIS_xy[0], CIS_xy[1])

In [ ]:
#################################
# Calculate and save density of first 0.5 m
################################

#Function to calculate the index of depth 0.5m
def depth05m(depth_array):
    """Return the position of the 0.5m"""
    if np.size(np.where(depth_array > 0.5)) == 0:
        return 0
    else:
        return np.where(depth_array > 0.5)[0][0]


# Calculate the 0.5m depth indices for all time steps
#print('Calculating depth index')
#depth05_ix = np.apply_along_axis(depth05m, axis=1, arr=sp_depth_past[:,:,:,:])
#depth05_ix = depth05m(sp_depth_past)

# Precompute constants
density_ice = 916.7
density_water = 997

depth = 0.5
filedens = sp_depth_past
fileVolI = sp_VolI_past
fileVolW = sp_VolW_past
savefilename = 'dens05m_past.npy'

# Initialize the output array
dens = np.zeros((filedens.shape[0], filedens.shape[2], filedens.shape[3]))

# Vectorized computation of density
t=0
for t in range(filedens.shape[0]):
    print('Calculating depth density')
    print(t)  # Progress indicator
    # Extract the relevant slices for the current time step
    #depth_indices = depth05_ix[t]
    vol_ice = fileVolI[t,:,:,:]
    vol_water = fileVolW[t,:,:,:]

    # Compute the density using vectorized operations
    for i in range(filedens.shape[2]):
        for j in range(filedens.shape[3]):
            depth_idx = np.apply_along_axis(depth05m, axis=0, arr=filedens[t,:,i,j])
            dens[t, i, j] = np.mean(
                vol_ice[:depth_idx, i, j] * density_ice +
                vol_water[:depth_idx, i, j] * density_water
            )

# Save as netcdf
np.save(savefilename, dens)
# -----------------------
#unout = 'datetime'
# =========================
#ncout = Dataset('/capstor/scratch/cscs/gsergi/dens05m_past_0.nc','w','NETCDF4'); # using netCDF3 for output format 
#ncout.createDimension('lon',np.size(lons,0));
#ncout.createDimension('lat',np.size(lats,1));
#ncout.createDimension('time',np.size(times));
#lonvar = ncout.createVariable('lons','float32',('lon','lat'));lonvar[:] = lons;
#latvar = ncout.createVariable('lats','float32',('lon','lat'));latvar[:] = lats;
#timevar = ncout.createVariable('time','float64',('time'));timevar.setncattr('units',unout);timevar[:]=times;
#dens05m_past = ncout.createVariable('dens05m_past','float32',('time','lat','lon'));dens05m_past.setncattr('units','kgm-2');dens05m_past[:] = dens05_past;
#ncout.close();

Calculating depth density
0
Calculating depth density
1
Calculating depth density
2
Calculating depth density
3
Calculating depth density
4
Calculating depth density
5
Calculating depth density
6
Calculating depth density
7
Calculating depth density
8
Calculating depth density
9
Calculating depth density
10
Calculating depth density
11
Calculating depth density
12
Calculating depth density
13
Calculating depth density
14
Calculating depth density
15
Calculating depth density
16
Calculating depth density
17
Calculating depth density
18
Calculating depth density
19
Calculating depth density
20
Calculating depth density
21
Calculating depth density
22
Calculating depth density
23
Calculating depth density
24
Calculating depth density
25
Calculating depth density
26
Calculating depth density
27
Calculating depth density
28
Calculating depth density
29
Calculating depth density
30
Calculating depth density
33
Calculating depth density
34
Calculating depth density
35
Calculating depth densit

In [ ]:
#################################
# Calculate and save density of first 0.5 m
################################

#Function to calculate the index of depth 0.5m
def depth05m(depth_array):
    """Return the position of the 0.5m"""
    if np.size(np.where(depth_array > 0.5)) == 0:
        return 0
    else:
        return np.where(depth_array > 0.5)[0][0]


# Calculate the 0.5m depth indices for all time steps
#print('Calculating depth index')
#depth05_ix = np.apply_along_axis(depth05m, axis=1, arr=sp_depth_past[:,:,:,:])
#depth05_ix = depth05m(sp_depth_past)

# Precompute constants
density_ice = 916.7
density_water = 997

depth = 0.5
filedens = sp_depth_hist
fileVolI = sp_VolI_hist
fileVolW = sp_VolW_hist
savefilename = 'dens05m_hist.npy'

# Initialize the output array
dens = np.zeros((filedens.shape[0], filedens.shape[2], filedens.shape[3]))

# Vectorized computation of density
t=0
for t in range(filedens.shape[0]):
    print('Calculating depth density')
    print(t)  # Progress indicator
    # Extract the relevant slices for the current time step
    #depth_indices = depth05_ix[t]
    vol_ice = fileVolI[t,:,:,:]
    vol_water = fileVolW[t,:,:,:]

    # Compute the density using vectorized operations
    for i in range(filedens.shape[2]):
        for j in range(filedens.shape[3]):
            depth_idx = np.apply_along_axis(depth05m, axis=0, arr=filedens[t,:,i,j])
            dens[t, i, j] = np.mean(
                vol_ice[:depth_idx, i, j] * density_ice +
                vol_water[:depth_idx, i, j] * density_water
            )

# Save as netcdf
np.save(savefilename, dens)
# -----------------------
#unout = 'datetime'
# =========================
#ncout = Dataset('/capstor/scratch/cscs/gsergi/dens05m_past_0.nc','w','NETCDF4'); # using netCDF3 for output format 
#ncout.createDimension('lon',np.size(lons,0));
#ncout.createDimension('lat',np.size(lats,1));
#ncout.createDimension('time',np.size(times));
#lonvar = ncout.createVariable('lons','float32',('lon','lat'));lonvar[:] = lons;
#latvar = ncout.createVariable('lats','float32',('lon','lat'));latvar[:] = lats;
#timevar = ncout.createVariable('time','float64',('time'));timevar.setncattr('units',unout);timevar[:]=times;
#dens05m_past = ncout.createVariable('dens05m_past','float32',('time','lat','lon'));dens05m_past.setncattr('units','kgm-2');dens05m_past[:] = dens05_past;
#ncout.close();

In [ ]:
def depth05m(depth_array):
    """Return the position of the 0.5m"""
    if np.size(np.where(depth_array > 0.5)) == 0:
        return 0
    else:
        return np.where(depth_array > 0.5)[0][0]

t=0

# Calculate the 0.5m depth indices for all time steps
#print('Calculating depth index')
#depth05_ix = np.apply_along_axis(depth05m, axis=1, arr=sp_depth_hist[:,:,:,:])
#depth05_ix = depth05m(sp_depth_hist)

# Precompute constants
density_ice = 916.7
density_water = 997

# Initialize the output array
dens05 = np.zeros((sp_depth_hist.shape[0], sp_depth_hist.shape[2], sp_depth_hist.shape[3]))

# Vectorized computation of density
for t in range(sp_depth_hist.shape[0]):
    print('Calculating depth density')
    print(t)  # Progress indicator
    # Extract the relevant slices for the current time step
    #depth_indices = depth05_ix[t]
    vol_ice = sp_VolI_hist[t,:,:,:]
    vol_water = sp_VolW_hist[t,:,:,:]

    # Compute the density using vectorized operations
    for i in range(sp_depth_hist.shape[2]):
        for j in range(sp_depth_hist.shape[3]):
            depth_idx = np.apply_along_axis(depth05m, axis=0, arr=sp_depth_hist[t,:,i,j])
            dens05[t, i, j] = np.mean(
                vol_ice[:depth_idx, i, j] * density_ice +
                vol_water[:depth_idx, i, j] * density_water
            )

In [ ]:
#!/usr/bin/env ipython
# ---------------------
import numpy as np
import datetime
from netCDF4 import Dataset,num2date,date2num
# -----------------------
unout = 'datetime'
# =========================
ncout = Dataset('/capstor/scratch/cscs/gsergi/dens05m_hist_0.nc','w','NETCDF4'); # using netCDF3 for output format 
ncout.createDimension('lon',np.size(lons,0));
ncout.createDimension('lat',np.size(lats,1));
ncout.createDimension('time',np.size(times));
lonvar = ncout.createVariable('lons','float32',('lon','lat'));lonvar[:] = lons;
latvar = ncout.createVariable('lats','float32',('lon','lat'));latvar[:] = lats;
timevar = ncout.createVariable('time','float64',('time'));timevar.setncattr('units',unout);timevar[:]=times;
dens05m_hist = ncout.createVariable('dens05m_hist','float32',('time','lat','lon'));dens05m_hist.setncattr('units','kgm-2');dens05m_hist[:] = dens05;
ncout.close();

In [ ]:
np.save('dens05m_hist.npy', dens05m_hist)
np.save('dens05m_past.npy', dens05m_past)